In [2]:
#install dependencies
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import os

Data augmentation: increases the diversity of your training data by applying random transformations (like rotations, flips, and zooms), which helps in improving model performance without needing more data.

Explanation: Data augmentation is applied to the training data to increase variability without the need for more images, helping the model generalize better.

Choosing class_mode

binary: For binary classification problems (two classes).

categorical: For multi-class classification problems (more than two classes) with one-hot encoded labels.

sparse: For multi-class classification problems with integer labels.

None: When you don’t need labels (only images).

Convolutional Networks (CNNs): Best for image-related tasks (e.g., object detection, image classification).

In [4]:
#Set the paths to your dataset
train_dir = '/content/drive/MyDrive/Image Identification/scraped_images'
#test_dir = '/content/drive/MyDrive/Image Identification/person.v5i.createml/test'
validation_dir = '/content/drive/MyDrive/Image Identification/Project_Images_1/data'

# Create an ImageDataGenerator object for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the images from the directories and label them
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=16, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=8, class_mode='binary')
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2))) # Add MaxPooling layer to the model
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2))) # Add MaxPooling layer to the model
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2))) # Add MaxPooling layer to the model
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2))) # Add MaxPooling layer to the model

model.add(layers.Flatten())
model.add(layers.Dense(512, activation ='relu'))
model.add(layers.Dense(1, activation='sigmoid') ) # Sigmoid for binary classificatio# Sigmoid for binary classification (mask vs. non-mask)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Image Identification/scraped_images'

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_generator, steps=10)
print(f'Validation Accuracy: {validation_accuracy:.2f}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9449 - loss: 0.1973
Validation Accuracy: 0.94


In [ ]:
model.save('mask_detector_model.keras')

In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load the saved model
model = load_model('mask_detector_model.keras')

# Step 1b: (Optional) Compile the model if you need to evaluate or continue training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Load and preprocess the image
img_path = '/content/drive/MyDrive/Image Identification/person.v5i.createml/valid/maksssksksss796_png.rf.e645ef8d30ce6a0b6d6cbb3f75b0e87e.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Step 3: Predict the class of the image
prediction = model.predict(img_array)

# Step 4: Visualize the image and prediction
plt.imshow(img)
plt.title("Mask" if prediction[0] <= 0.5 else "Non-Mask")
plt.axis('off')
plt.show()
